# Local ```minio``` R/W through ```fsspec```.

"*Local*" for this example is on the local machine this

This notebook will have examples on using a locally hosted minio instance (Such as by the one started in [98_minio](98_minio.ipynb)).

- Setup:
  1. Run minio from [98_minio](98_minio.ipynb). TODO: Make a subprocess tool to handle this.
  2. Find the ```secret```, ```key``` and ```url``` for your company's minio instance.  
    Update the information in the ```s3_cfg``` dict.

# ```minio``` Setup.

In addition to the ```key``` & ```secret``` from AWS, you can all specify an ```endpoint_url```.

In [44]:
s3_cfg = {
    "key": "minio",
    "secret": "miniostorage",
    "client_kwargs": {
        "endpoint_url": "http://127.0.0.1:9000",
    },
}

import fsspec
fs = fsspec.filesystem("s3", **s3_cfg)

List the buckets:

In [46]:
fs.ls("")

['mdfbucket-0',
 'mdfbucket-1',
 'mdfbucket-2',
 'mdfbucket-3',
 'mdfbucket-4',
 'mdfbucket-5',
 'mdfbucket-6',
 'mdfbucket-7',
 'mdfbucket-8',
 'mdfbucket-9']

Make 10 buckets (if they don't already exist):

In [47]:
for bucket in range(10):
    try:
        bucket=f"mdfbucket-{bucket}"
        fs.mkdir(bucket)
    except FileExistsError:
        pass    

In [48]:
fs.ls("")

['mdfbucket-0',
 'mdfbucket-1',
 'mdfbucket-2',
 'mdfbucket-3',
 'mdfbucket-4',
 'mdfbucket-5',
 'mdfbucket-6',
 'mdfbucket-7',
 'mdfbucket-8',
 'mdfbucket-9']

Loop through and write a 'hello.txt' to each of the buckets.

In [49]:
for bucket in fs.ls(""):
    with fs.open(f"{bucket}/hello.txt", "wt") as fid:
        print(f"Hello {bucket}", file=fid)

# Read
Read the file back through both the local file system:

In [56]:
with open("mdf_data/mdfbucket-0/hello.txt", "rt") as fid:
    print(fid.read())

Hello mdfbucket-0



Read the file back through S3.

In [57]:
with fs.open("mdfbucket-0/hello.txt", "rt") as fid:
    print(fid.read())

Hello mdfbucket-0



# Walk

Walk through the file system looking for files.

Walk through the local file system with ```os```:

In [72]:
import os
for root, directories, files in os.walk("mdf_data/"):
    for file in files:
        if file.endswith(".txt"):
            full_path = os.path.join(root, file)
            with open(full_path, "rt") as fid:
                print(f"{file}:")
                print(f"\t{fid.read()}\n")

hello.txt:
	Hello mdfbucket-1


hello.txt:
	Hello mdfbucket-6


hello.txt:
	Hello mdfbucket-2


hello.txt:
	Hello mdfbucket-8


hello.txt:
	Hello mdfbucket-5


hello.txt:
	Hello mdfbucket-7


hello.txt:
	Hello mdfbucket-0


hello.txt:
	Hello mdfbucket-4


hello.txt:
	Hello mdfbucket-9


hello.txt:
	Hello mdfbucket-3




Walk through the remote S3 file system with ```fsspec```:

In [73]:
for bucket in fs.ls(""):
    for root, directories, files in fs.walk(bucket):
        for file in files:
            full_path = os.path.join(root, file)
            with fs.open(full_path, "rt") as fid:
                print(f"{file}:")
                print(f"\t{fid.read()}\n")

hello.txt:
	Hello mdfbucket-0


hello.txt:
	Hello mdfbucket-1


hello.txt:
	Hello mdfbucket-2


hello.txt:
	Hello mdfbucket-3


hello.txt:
	Hello mdfbucket-4


hello.txt:
	Hello mdfbucket-5


hello.txt:
	Hello mdfbucket-6


hello.txt:
	Hello mdfbucket-7


hello.txt:
	Hello mdfbucket-8


hello.txt:
	Hello mdfbucket-9


